# Baseline MNIST FCN

In [1]:
import tensorflow as tf
import numpy as np
import os
from scipy.special import softmax

In [2]:
import vbranch

In [3]:
save = True
model_id = 5

## Load Data

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
input_dim = 784
num_classes = 10

In [6]:
X_train_flat = X_train.reshape([-1, input_dim])
X_test_flat = X_test.reshape([-1, input_dim])

y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

## Build Model

In [7]:
BATCH_SIZE = 32
EPOCHS = 10
STEPS_PER_EPOCH = 100
model_path = './models/mnist_' + str(model_id)

In [8]:
tf.reset_default_graph()

train_data = (X_train_flat.astype('float32'), y_train_one_hot)
test_data = (X_test_flat.astype('float32'), y_test_one_hot)

batch_size = tf.placeholder('int64', name='batch_size')

train_dataset = tf.data.Dataset.from_tensor_slices(train_data).\
    batch(batch_size).repeat().\
    shuffle(buffer_size=4*BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices(test_data).\
    batch(batch_size).repeat()

iter_ = tf.data.Iterator.from_structure(train_dataset.output_types, 
                                       train_dataset.output_shapes)
inputs, labels_one_hot = iter_.get_next()

train_init_op = iter_.make_initializer(train_dataset)
test_init_op = iter_.make_initializer(test_dataset, name='test_init_op')

In [9]:
with tf.variable_scope('model_' + str(model_id)):
    model = vbranch.models.simple_fcn(inputs, 128, num_classes)

In [10]:
model.summary()

i   Layer name          Output shape        Parameters                    Num param 
------------------------------------------------------------------------------------
    Input               [None,784]                                                  
------------------------------------------------------------------------------------
0   fc1                 [None,128]          (784,128) (128,)              100480    
------------------------------------------------------------------------------------
1   bn1                 [None,128]          (128,) (128,)                 256       
------------------------------------------------------------------------------------
2   relu1               [None,128]                                        0         
------------------------------------------------------------------------------------
3   fc2                 [None,10]           (128,10) (10,)                1290      
-----------------------------------------------------------------

In [11]:
tf.global_variables()

[<tf.Variable 'model_5/fc1_w:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable 'model_5/fc1_b:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'model_5/bn1_scale:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'model_5/bn1_beta:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'model_5/fc2_w:0' shape=(128, 10) dtype=float32_ref>,
 <tf.Variable 'model_5/fc2_b:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'model_5/bn2_scale:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'model_5/bn2_beta:0' shape=(10,) dtype=float32_ref>]

In [12]:
loss = vbranch.losses.softmax_cross_entropy_with_logits(labels=labels_one_hot, 
                                                        logits=model.output, name='loss')
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [13]:
pred = tf.nn.softmax(model.output, name='pred')
pred_max = tf.one_hot(tf.argmax(pred, axis=-1), num_classes)
acc = tf.reduce_mean(tf.reduce_sum(labels_one_hot*pred_max, [1]), name='acc')

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for e in range(EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        progbar = tf.keras.utils.Progbar(STEPS_PER_EPOCH)
        
        sess.run(train_init_op, feed_dict={batch_size: BATCH_SIZE})

        for i in range(STEPS_PER_EPOCH):
            _, loss_value, acc_value = sess.run([train_op, loss, acc])
            
            if i == STEPS_PER_EPOCH - 1:
                sess.run(test_init_op, feed_dict={batch_size: len(X_test_flat)})
                val_loss, val_acc = sess.run([loss, acc])
                progbar.update(i + 1, values=[("loss", loss_value), ("acc", acc_value), 
                                              ("val_loss", val_loss), ("val_acc", val_acc)])
            else:
                progbar.update(i + 1, values=[("loss", loss_value), ("acc", acc_value)])
    
    if save:
        saver = tf.train.Saver()
        path = os.path.join(model_path, 'ckpt')
        saver.save(sess, path)

Epoch 1/10
100/100 [==============================] - 14s 142ms/step - loss: 1.1410 - acc: 0.7528 - val_loss: 0.7710 - val_acc: 0.8908
Epoch 2/10
100/100 [==============================] - 0s 4ms/step - loss: 0.7501 - acc: 0.8884 - val_loss: 0.6236 - val_acc: 0.9151
Epoch 3/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6208 - acc: 0.9150 - val_loss: 0.5414 - val_acc: 0.9252
Epoch 4/10
100/100 [==============================] - 0s 4ms/step - loss: 0.5267 - acc: 0.9372 - val_loss: 0.4776 - val_acc: 0.9322
Epoch 5/10
100/100 [==============================] - 0s 4ms/step - loss: 0.4713 - acc: 0.9409 - val_loss: 0.4300 - val_acc: 0.9345
Epoch 6/10
100/100 [==============================] - 0s 4ms/step - loss: 0.4047 - acc: 0.9547 - val_loss: 0.3903 - val_acc: 0.9401
Epoch 7/10
100/100 [==============================] - 0s 4ms/step - loss: 0.3547 - acc: 0.9672 - val_loss: 0.3687 - val_acc: 0.9404
Epoch 8/10
100/100 [==============================] - 0s 4ms/step - loss:

## Model Ensemble

In [16]:
test_outputs = []
test_losses = []
test_accs = []

num_models = 2
graphs = [tf.Graph() for _ in range(num_models)]
sessions = [tf.Session(graph=g) for g in graphs]

for i in range(len(graphs)):
    with graphs[i].as_default():
        model_path = './models/mnist_' + str(i + 1)
        meta_path = os.path.join(model_path, 'ckpt.meta')
        ckpt = tf.train.get_checkpoint_state(model_path)
        
        imported_graph = tf.train.import_meta_graph(meta_path)
        imported_graph.restore(sessions[i], ckpt.model_checkpoint_path)
                
        sessions[i].run('test_init_op', feed_dict={'batch_size:0': len(X_test_flat)})
        
        output, loss, acc = sessions[i].run(['model_%d'%(i+1)+'/'+'output:0', 
                                             'loss:0', 'acc:0'])
        test_outputs.append(output)
        test_losses.append(loss)
        test_accs.append(acc)

INFO:tensorflow:Restoring parameters from ./models/mnist_1/ckpt
INFO:tensorflow:Restoring parameters from ./models/mnist_2/ckpt


In [18]:
test_accs

[0.9472, 0.9465]

In [19]:
def compute_acc(pred, labels_one_hot):
    pred_max = tf.keras.utils.to_categorical(np.argmax(pred, axis=-1), num_classes)
    return np.mean(np.sum(labels_one_hot*pred_max, axis=1))

In [22]:
for i in range(num_models):
    assert compute_acc(test_outputs[i], y_test_one_hot) == test_accs[i]

### Average Predictions Before Softmax

In [23]:
before_mean_output = np.array(test_outputs).mean(axis=0)
before_mean_acc = compute_acc(softmax(before_mean_output, axis=-1), y_test_one_hot)

In [24]:
print(before_mean_acc)

0.9512


### Average Predictions After Softmax

In [25]:
after_mean_output = softmax(np.array(test_outputs), axis=-1).mean(axis=0)
after_mean_acc = compute_acc(softmax(after_mean_output, axis=-1), y_test_one_hot)

In [26]:
print(after_mean_acc)

0.951
